# Ridge and Lassso Regression

[Resource](https://harvard-iacs.github.io/2017-CS109A/lectures/lecture7/notebook/)

Nice lil Harvard University resource here for ya. Who needs a masters! (Just kidding, I probably will at some point).

Let's get these bajillion imports out of the way.

In [ ]:
import pandas as pd
import sys
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.tools import add_constant
from statsmodels.regression.linear_model import RegressionResults
import seaborn as sns
import sklearn as sk
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold
from sklearn.linear_model import 

In [1]:
import statsmodels.api as sm

print(sm.OLS.fit_regularized.__doc__)


Return a regularized fit to a linear regression model.

Parameters
----------
method : str
    Either 'elastic_net' or 'sqrt_lasso'.
alpha : scalar or array_like
    The penalty weight.  If a scalar, the same penalty weight
    applies to all variables in the model.  If a vector, it
    must have the same length as `params`, and contains a
    penalty weight for each coefficient.
L1_wt : scalar
    The fraction of the penalty given to the L1 penalty term.
    Must be between 0 and 1 (inclusive).  If 0, the fit is a
    ridge fit, if 1 it is a lasso fit.
start_params : array_like
    Starting values for ``params``.
profile_scale : bool
    If True the penalized fit is computed using the profile
    (concentrated) log-likelihood for the Gaussian model.
    Otherwise the fit uses the residual sum of squares.
refit : bool
    If True, the model is refit using only the variables that
    have non-zero coefficients in the regularized fit.  The
    refitted model is not regularized.
**kwargs